# Exploration of ONS API

Test how you use the API and locate the right files.

In [3]:
#imports
import pandas as pd
import requests

In [5]:
#base url
endpoint = "https://api.beta.ons.gov.uk/v1"

In [ ]:
#get datasets

dataset_resp = requests.get(endpoint + "/datasets")
dataset_json = dataset_resp.json()

dataset_df = pd.DataFrame(dataset_json)

dataset_df.head()

,items,count,offset,limit,total_count
0,{'contacts': [{'email': 'qualityoflife@ons.gov...,20,0,20,337
1,{'contacts': [{'email': 'QualityOfLife@ons.gov...,20,0,20,337
2,{'contacts': [{'email': 'health.data@ons.gov.u...,20,0,20,337
3,{'contacts': [{'email': 'health.data@ons.gov.u...,20,0,20,337
4,{'contacts': [{'email': 'health.data@ons.gov.u...,20,0,20,337


In the above cell you got 20 results, but the `total_count` column implies there are 337 datasets. How do we get them all?
[API docs](https://developer.ons.gov.uk/dataset/datasets/) suggest we can use a `limit` argument to get all results.

In [ ]:
params = {"limit": 337}

dataset_resp = requests.get(endpoint + "/datasets", params = params)
dataset_json = dataset_resp.json()

dataset_df = pd.DataFrame(dataset_json) 

dataset_df.shape[0]

337

337 rows! More promising. Let's take a look

In [29]:
dataset_df.head()

,items,count,offset,limit,total_count
0,{'contacts': [{'email': 'qualityoflife@ons.gov...,337,0,337,337
1,{'contacts': [{'email': 'QualityOfLife@ons.gov...,337,0,337,337
2,{'contacts': [{'email': 'health.data@ons.gov.u...,337,0,337,337
3,{'contacts': [{'email': 'health.data@ons.gov.u...,337,0,337,337
4,{'contacts': [{'email': 'health.data@ons.gov.u...,337,0,337,337


We're clearly interested in the `items` column - we'll create the dataframe on that instead.

In [30]:
items_df = pd.DataFrame(dataset_json["items"])

items_df.head()

,contacts,description,keywords,id,last_updated,links,methodologies,national_statistic,next_release,qmi,...,title,unit_of_measure,type,publications,license,is_based_on,canonical_topic,subtopics,survey,related_content
0,"[{'email': 'qualityoflife@ons.gov.uk', 'name':...",Seasonally and non seasonally-adjusted quarter...,[well-being],wellbeing-quarterly,2023-12-13T09:40:24.204Z,{'editions': {'href': 'https://api.beta.ons.go...,[{'href': 'https://www.ons.gov.uk/peoplepopula...,False,TBC,{'href': 'https://www.ons.gov.uk/peoplepopulat...,...,Quarterly personal well-being estimates,Percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'email': 'QualityOfLife@ons.gov.uk', 'name':...","Estimates of life satisfaction, feeling that t...",[well-being],wellbeing-local-authority,2023-12-13T09:40:21.928Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,TBC,{'href': 'https://www.ons.gov.uk/peoplepopulat...,...,Personal well-being estimates by local authority,Percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'email': 'health.data@ons.gov.uk', 'name': '...",Provisional counts of the number of deaths reg...,"[Deaths, weekly deaths]",weekly-deaths-region,2025-06-05T09:11:16.664Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,11 June 2025,{'href': 'https://www.ons.gov.uk/peoplepopulat...,...,Deaths registered weekly in England and Wales ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'email': 'health.data@ons.gov.uk', 'name': '...",Provisional counts of the number of deaths reg...,"[Deaths, weekly deaths]",weekly-deaths-local-authority,2024-01-09T10:10:57.755Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,17 January 2024,{'href': 'https://www.ons.gov.uk/peoplepopulat...,...,Death registrations and occurrences by local a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'email': 'health.data@ons.gov.uk', 'name': '...",Provisional counts of the number of deaths reg...,"[Deaths, weekly deaths]",weekly-deaths-health-board,2024-01-09T10:11:08.441Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,17 January 2024,{'href': 'https://www.ons.gov.uk/peoplepopulat...,...,Death registrations and occurrences by health ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A good bet for finding what we're after is to explode the `keywords` column and look for "ashe" or "earnings"...

In [32]:
#unnest keywords and see if that can identify ashe
unnested = items_df.explode("keywords")

ashe_check = unnested[unnested["keywords"].str.contains("ashe|earnings", case = False, na = False)]

ashe_check.head()

,contacts,description,keywords,id,last_updated,links,methodologies,national_statistic,next_release,qmi,...,title,unit_of_measure,type,publications,license,is_based_on,canonical_topic,subtopics,survey,related_content
39,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-9-and-10,2024-01-25T09:40:12.013Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,...,"Earnings and hours worked, place of work and p...",NaN,NaN,[{'href': 'https://www.ons.gov.uk/employmentan...,NaN,NaN,NaN,NaN,NaN,NaN
39,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,"pay,earnings,hours",ashe-tables-9-and-10,2024-01-25T09:40:12.013Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,...,"Earnings and hours worked, place of work and p...",NaN,NaN,[{'href': 'https://www.ons.gov.uk/employmentan...,NaN,NaN,NaN,NaN,NaN,NaN
40,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-7-and-8,2024-01-23T09:47:44.212Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,...,"Earnings and hours worked, place of work and r...",NaN,NaN,[{'href': 'https://www.ons.gov.uk/releases/emp...,NaN,NaN,NaN,NaN,NaN,NaN
41,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-3,2024-01-23T09:47:39.729Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,...,"Earnings and hours worked, region by occupatio...",NaN,NaN,[{'href': 'https://www.ons.gov.uk/employmentan...,NaN,NaN,NaN,NaN,NaN,NaN
43,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-26,2024-01-23T09:47:37.163Z,{'editions': {'href': 'https://api.beta.ons.go...,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,...,"Earnings and hours worked, care workers: ASHE ...",NaN,NaN,[{'href': 'https://www.ons.gov.uk/employmentan...,NaN,NaN,NaN,NaN,NaN,NaN


Looks promising! But we need to find how we download the actual datasets. Maybe the `links` field is a good bet?

In [ ]:
links = ashe_check["links"].apply(pd.Series) #turn links dict into a df

links_df = pd.concat([ashe_check.drop(columns = "links"), links], axis = 1) #replace links dict with above df

links_df.head()

,contacts,description,keywords,id,last_updated,methodologies,national_statistic,next_release,qmi,related_datasets,...,license,is_based_on,canonical_topic,subtopics,survey,related_content,editions,latest_version,self,taxonomy
39,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-9-and-10,2024-01-25T09:40:12.013Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...
39,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,"pay,earnings,hours",ashe-tables-9-and-10,2024-01-25T09:40:12.013Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...
40,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-7-and-8,2024-01-23T09:47:44.212Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...
41,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-3,2024-01-23T09:47:39.729Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...
43,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-26,2024-01-23T09:47:37.163Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...


`editions` seems a good place to look....

In [37]:
editions = links_df["editions"].apply(pd.Series)

editions_df = pd.concat([links_df.drop(columns = "editions"), editions], axis = 1)

editions_df.head()

,contacts,description,keywords,id,last_updated,methodologies,national_statistic,next_release,qmi,related_datasets,...,license,is_based_on,canonical_topic,subtopics,survey,related_content,latest_version,self,taxonomy,href
39,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-9-and-10,2024-01-25T09:40:12.013Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...,https://api.beta.ons.gov.uk/v1/datasets/ashe-t...
39,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,"pay,earnings,hours",ashe-tables-9-and-10,2024-01-25T09:40:12.013Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...,https://api.beta.ons.gov.uk/v1/datasets/ashe-t...
40,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-7-and-8,2024-01-23T09:47:44.212Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...,https://api.beta.ons.gov.uk/v1/datasets/ashe-t...
41,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-3,2024-01-23T09:47:39.729Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...,https://api.beta.ons.gov.uk/v1/datasets/ashe-t...
43,"[{'email': 'earnings@ons.gov.uk', 'name': 'Nic...",Annual estimates of paid hours worked and earn...,ASHE,ashe-tables-26,2024-01-23T09:47:37.163Z,NaN,True,To be announced,{'href': 'https://www.ons.gov.uk/employmentand...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/emplo...,https://api.beta.ons.gov.uk/v1/datasets/ashe-t...


Now we have a `href` column - presumably a link to find out more information about editions. Let's see what happens if we send a request to one of these for the API.

In [40]:
test_href = editions_df["href"].tolist()[0]

href_request = requests.get(test_href)
href_json = href_request.json()

href_df = pd.DataFrame(href_json)
href_df.head()

,items,count,offset,limit,total_count
0,"{'edition': 'time-series', 'id': '1009955f-fec...",1,0,20,1


Again we want `items`

In [41]:
href_df = pd.DataFrame(href_json["items"])
href_df.head()

,edition,id,links,state
0,time-series,1009955f-feca-4abb-8699-7cb10baecefe,{'dataset': {'href': 'https://api.beta.ons.gov...,published


...and it seems we now want to extract data from the `links` column again

In [42]:
links = href_df["links"].apply(pd.Series)

links_df = pd.concat([href_df.drop(columns = "links"), links], axis = 1)

links_df.head()

,edition,id,state,dataset,latest_version,self,versions
0,time-series,1009955f-feca-4abb-8699-7cb10baecefe,published,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...


we want a history, so probably want to access the `versions`

In [43]:
versions = links_df["versions"].apply(pd.Series)

versions_df = pd.concat([links_df.drop(columns = "versions"), versions], axis = 1)
versions_df.head()

,edition,id,state,dataset,latest_version,self,href
0,time-series,1009955f-feca-4abb-8699-7cb10baecefe,published,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,{'href': 'https://api.beta.ons.gov.uk/v1/datas...,https://api.beta.ons.gov.uk/v1/datasets/ashe-t...


and we've got another link to follow - let's make a request to it.

In [75]:
test_href = versions_df["href"].tolist()[0]

href_request = requests.get(test_href)
href_json = href_request.json()

href_df = pd.DataFrame(href_json)
href_df.head()

,items,count,offset,limit,total_count
0,"{'alerts': [], 'collection_id': 'cmdashetables...",7,0,20,7
1,"{'alerts': [], 'collection_id': 'cmdashetables...",7,0,20,7
2,"{'alerts': [], 'collection_id': 'cmdashetables...",7,0,20,7
3,"{'alerts': [], 'collection_id': 'cmdashe9and10...",7,0,20,7
4,"{'alerts': [], 'collection_id': 'cmdashetables...",7,0,20,7


and we want items again

In [76]:
href_df = pd.DataFrame(href_json["items"])
href_df.head()

,alerts,collection_id,dataset_id,dimensions,downloads,edition,id,last_updated,latest_changes,links,release_date,state,usage_notes,version,type
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4
1,[],cmdashetables9and10-d765eb175c07a1121de05a132e...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,dda31a80-a12d-40cd-a634-04cf827a652e,2024-01-23T09:47:13.276Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-22T00:00:00.000Z,published,[],6,v4
2,[],cmdashetables9and10-052b867dcc51c294df33e92a8a...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",{},time-series,3a58a7e1-4e69-4686-8ee7-542a36195163,2023-09-04T09:19:41.298Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2023-09-04T00:00:00.000Z,published,[],5,v4
3,[],cmdashe9and10-f07ea6865e08883c405bd348a11124e8...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,NaN,time-series,9070e83e-6bb5-4407-8866-78f869d4316b,2022-11-16T10:10:01.433Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2022-11-07T00:00:00.000Z,published,[],4,v4
4,[],cmdashetables9and10-bb6c38abc7dd2247e14913a16b...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,6c86d122-1cc8-46c2-af15-c39aa2518baa,2021-11-26T10:29:23.34Z,NaN,{'dataset': {'href': 'https://api.beta.ons.gov...,2021-11-01T00:00:00.000Z,published,[{'title': 'ASHE covers employee jobs in the U...,3,v4


## Exploring dimensions

we should probably take a look at the `dimensions` column

In [47]:
dimensions_df = href_df.explode("dimensions")

dimensions = dimensions_df["dimensions"].apply(pd.Series)
dimensions_df = pd.concat([dimensions_df.drop(columns = "dimensions"), dimensions], axis = 1)

dimensions_df.head()

,alerts,collection_id,dataset_id,downloads,edition,id,last_updated,latest_changes,links,release_date,state,usage_notes,version,type,label,links,href,id,name,description
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4,Time,"{'code_list': {}, 'options': {}, 'version': {}}",https://api.beta.ons.gov.uk/v1/code-lists/cale...,calendar-years,time,NaN
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4,Geography,"{'code_list': {}, 'options': {}, 'version': {}}",https://api.beta.ons.gov.uk/v1/code-lists/parl...,parliamentary-constituencies,geography,NaN
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4,Averages and percentiles,"{'code_list': {}, 'options': {}, 'version': {}}",https://api.beta.ons.gov.uk/v1/code-lists/aver...,averages-and-percentiles,averagesandpercentiles,NaN
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4,Sex,"{'code_list': {}, 'options': {}, 'version': {}}",https://api.beta.ons.gov.uk/v1/code-lists/sex,sex,sex,NaN
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4,Working pattern,"{'code_list': {}, 'options': {}, 'version': {}}",https://api.beta.ons.gov.uk/v1/code-lists/work...,working-pattern,workingpattern,NaN


it looks like we have links to where we can access dimensional tables - this will be useful in addition to the 'core' datasets (which will likely be more like fact tables). Let's explore one.

In [48]:
test_dimension = dimensions_df["href"].tolist()[0] #we know this one is time from above

time_request = requests.get(test_dimension)
time_json = time_request.json()

time_df = pd.DataFrame(time_json)
time_df.head()

,links
self,"{'id': 'calendar-years', 'href': 'http://api.b..."
editions,{'href': 'http://api.beta.ons.gov.uk/v1/code-l...


I thought we'd just want the details in `self` initially, but actually seems we want what's in `editions`...

In [59]:
editions_df = time_df.loc[["editions"]]

links = editions_df["links"].apply(pd.Series)
editions_df = pd.concat([editions_df.drop(columns = "links"), links], axis = 1)
editions_df

,href
editions,http://api.beta.ons.gov.uk/v1/code-lists/calen...


In [61]:
year_href = editions_df["href"].tolist()[0] #we know this one is time from above

year_request = requests.get(year_href)
year_json = year_request.json()

year_df = pd.DataFrame(year_json["items"]) # of course we want items
year_df.head()

,edition,label,links
0,one-off,,"{'self': {'id': 'one-off', 'href': 'http://api..."


In [ ]:
year_df["links"][0]

{'self': {'id': 'one-off',
  'href': 'http://api.beta.ons.gov.uk/v1/code-lists/calendar-years/editions/one-off'},
 'editions': {'href': 'http://api.beta.ons.gov.uk/v1/code-lists/calendar-years/editions'},
 'codes': {'href': 'http://api.beta.ons.gov.uk/v1/code-lists/calendar-years/editions/one-off/codes'}}

In [71]:
year_codes = year_df["links"][0]["codes"]["href"]

In [ ]:
codes_request = requests.get(year_codes)
codes_json = codes_request.json()

codes_df = pd.DataFrame(codes_json)
codes_df.head()

,items,count,offset,limit,total_count
0,"{'code': '2150', 'label': '2150', 'links': {'c...",20,0,20,201
1,"{'code': '2149', 'label': '2149', 'links': {'c...",20,0,20,201
2,"{'code': '2148', 'label': '2148', 'links': {'c...",20,0,20,201
3,"{'code': '2147', 'label': '2147', 'links': {'c...",20,0,20,201
4,"{'code': '2146', 'label': '2146', 'links': {'c...",20,0,20,201


In [74]:
params = {"limit": 201}

codes_request = requests.get(year_codes, params = params)
codes_json = codes_request.json()

codes_df = pd.DataFrame(codes_json["items"])
codes_df

,code,label,links
0,2150,2150,{'code_list': {'href': 'http://api.beta.ons.go...
1,2149,2149,{'code_list': {'href': 'http://api.beta.ons.go...
2,2148,2148,{'code_list': {'href': 'http://api.beta.ons.go...
3,2147,2147,{'code_list': {'href': 'http://api.beta.ons.go...
4,2146,2146,{'code_list': {'href': 'http://api.beta.ons.go...
...,...,...,...
196,1954,1954,{'code_list': {'href': 'http://api.beta.ons.go...
197,1953,1953,{'code_list': {'href': 'http://api.beta.ons.go...
198,1952,1952,{'code_list': {'href': 'http://api.beta.ons.go...
199,1951,1951,{'code_list': {'href': 'http://api.beta.ons.go...


Halleleujah! This would make for a useful dimensional table in the model, so we should set these steps up as a function to get the relevant data needed for those.

Now to go back to getting the obervations...

## Downloading observations
go back to `href_df`

In [77]:
href_df.head()

,alerts,collection_id,dataset_id,dimensions,downloads,edition,id,last_updated,latest_changes,links,release_date,state,usage_notes,version,type
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4
1,[],cmdashetables9and10-d765eb175c07a1121de05a132e...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,dda31a80-a12d-40cd-a634-04cf827a652e,2024-01-23T09:47:13.276Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-22T00:00:00.000Z,published,[],6,v4
2,[],cmdashetables9and10-052b867dcc51c294df33e92a8a...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",{},time-series,3a58a7e1-4e69-4686-8ee7-542a36195163,2023-09-04T09:19:41.298Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2023-09-04T00:00:00.000Z,published,[],5,v4
3,[],cmdashe9and10-f07ea6865e08883c405bd348a11124e8...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,NaN,time-series,9070e83e-6bb5-4407-8866-78f869d4316b,2022-11-16T10:10:01.433Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2022-11-07T00:00:00.000Z,published,[],4,v4
4,[],cmdashetables9and10-bb6c38abc7dd2247e14913a16b...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,{'csv': {'href': 'https://download.ons.gov.uk/...,time-series,6c86d122-1cc8-46c2-af15-c39aa2518baa,2021-11-26T10:29:23.34Z,NaN,{'dataset': {'href': 'https://api.beta.ons.gov...,2021-11-01T00:00:00.000Z,published,[{'title': 'ASHE covers employee jobs in the U...,3,v4


we presumably want the `downloads`  

In [78]:
downloads = href_df["downloads"].apply(pd.Series)

href_df = pd.concat([href_df.drop(columns = "downloads"), downloads], axis = 1)
href_df

,alerts,collection_id,dataset_id,dimensions,edition,id,last_updated,latest_changes,links,release_date,state,usage_notes,version,type,csv,csvw,0
0,[],cmdashetables9and10-1e517c5ad8ea697cdbe71c94b5...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",time-series,a4f2f661-312b-4d39-9d7e-f8ca46bd22af,2024-01-25T09:40:05.702Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-24T00:00:00.000Z,published,[],7,v4,{'href': 'https://download.ons.gov.uk/download...,{'href': 'https://download.ons.gov.uk/download...,NaN
1,[],cmdashetables9and10-d765eb175c07a1121de05a132e...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",time-series,dda31a80-a12d-40cd-a634-04cf827a652e,2024-01-23T09:47:13.276Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2024-01-22T00:00:00.000Z,published,[],6,v4,{'href': 'https://download.ons.gov.uk/download...,{'href': 'https://download.ons.gov.uk/download...,NaN
2,[],cmdashetables9and10-052b867dcc51c294df33e92a8a...,ashe-tables-9-and-10,"[{'label': 'Time', 'links': {'code_list': {}, ...",time-series,3a58a7e1-4e69-4686-8ee7-542a36195163,2023-09-04T09:19:41.298Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2023-09-04T00:00:00.000Z,published,[],5,v4,NaN,NaN,NaN
3,[],cmdashe9and10-f07ea6865e08883c405bd348a11124e8...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,time-series,9070e83e-6bb5-4407-8866-78f869d4316b,2022-11-16T10:10:01.433Z,[],{'dataset': {'href': 'https://api.beta.ons.gov...,2022-11-07T00:00:00.000Z,published,[],4,v4,NaN,NaN,NaN
4,[],cmdashetables9and10-bb6c38abc7dd2247e14913a16b...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,time-series,6c86d122-1cc8-46c2-af15-c39aa2518baa,2021-11-26T10:29:23.34Z,NaN,{'dataset': {'href': 'https://api.beta.ons.gov...,2021-11-01T00:00:00.000Z,published,[{'title': 'ASHE covers employee jobs in the U...,3,v4,{'href': 'https://download.ons.gov.uk/download...,{'href': 'https://download.ons.gov.uk/download...,NaN
5,[],cmdashetables9and10-599526a89390769db60eecfe4c...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,time-series,63c7ce8a-c528-4e5f-87e3-ebf1dc6a1113,2021-02-08T12:54:45.368Z,NaN,{'dataset': {'href': 'https://api.beta.ons.gov...,2020-11-03T00:00:00.000Z,published,[{'title': 'ASHE covers employee jobs in the U...,2,NaN,{'href': 'https://download.ons.gov.uk/download...,{'href': 'https://download.ons.gov.uk/download...,NaN
6,[],cmdashe9and10-6d8286c3d9024ba355749613aa8c55de...,ashe-tables-9-and-10,[{'description': 'The latest year has provisio...,time-series,aa66ebc8-eb48-450b-b407-25469bfc3723,2020-11-04T09:41:29.738Z,NaN,{'dataset': {'href': 'https://api.beta.ons.gov...,2019-10-29T00:00:00.000Z,published,[],1,NaN,NaN,NaN,NaN


...and we'll take the csv.

From trying, you can't directly read the csv to pandas, so we'd need to download first.
This takes a while so we'll want to invest in using `tqdm` in this project....

In [83]:
csv_href = href_df["csv"][0]["href"]
save_path = "test-downloads/observations-test.csv"

csv_response = requests.get(csv_href)
with open(save_path, "wb") as f:
    f.write(csv_response.content)

KeyboardInterrupt: 